In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# utilizado para la manipulación de directorios y rutas
import os
# Cálculo científico y vectorial para python
import numpy as np
# Libreria para graficos
from matplotlib import pyplot as plt


import torch
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader  # Gives easier dataset managment by creating mini batches etc.

import torchvision # torch package for vision related things
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
from tqdm import tqdm  # For nice progress bar!

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

In [5]:
# Definir la clase de la red neuronal
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

class RedNeuronalBinaria(nn.Module):
    def __init__(self, input_size):
        super(RedNeuronalBinaria, self).__init__()
        self.fc1 = nn.Linear(input_size, 64) #primera capa donde entra las caracterisitcas
        self.fc2 = nn.Linear(64, 32)#capa oculta
        self.fc3 = nn.Linear(32, 1)#capa de salida para saber si o no

    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))#se aplica la sgimoide a lasalida de las caracteristicas
        x = torch.sigmoid(self.fc2(x))# de nuevo se aplica la sigmoide
        x = torch.sigmoid(self.fc3(x))# y otra ves la sigmoide
        return x

# Cargo  mi data set binario para saber si es si o no separando con 80% entramiento y 20% prueba
data = np.loadtxt('/content/drive/MyDrive/Red Neuronal/preparado_1.2.txt', delimiter=';')

X = data[:, :-1]
y = data[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar características
mu_train = np.mean(X_train, axis=0)
sigma_train = np.std(X_train, axis=0)
sigma_train[sigma_train == 0] = 1e-10
X_train_norm = (X_train - mu_train) / sigma_train
X_test_norm = (X_test - mu_train) / sigma_train

# Convertir a tensores
X_train_tensor = torch.tensor(X_train_norm, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)  # Ajustar forma para una salida binaria
X_test_tensor = torch.tensor(X_test_norm, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)  # Ajustar forma para una salida binaria

# Crear conjuntos de datos PyTorch
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Tamaño de entrada del modelo (55 características en caso)
input_size = 55

# Número de clases (en clasificación binaria es 1)
num_classes = 1

# Coeficiente de aprendizaje
learning_rate = 0.001

# Tamaño del lote
batch_size = 64  # Puedes ajustar este valor según tus necesidades y el tamaño de tus datos

# Número de épocas (iteraciones completas sobre el conjunto de datos)
num_epochs = 10  # Puedes ajustar este valor según la convergencia del modelo y el tiempo de entrenamiento

# Crear cargadores de lotes
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Verificar si CUDA está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Dispositivo utilizado:", device)

# Crear una instancia del modelo
model = RedNeuronalBinaria(input_size=input_size).to(device)

# Función de pérdida y optimizador
criterion = nn.BCELoss()  # Para una salida binaria
optimizer = optim.Adam(model.parameters(), lr=learning_rate)#optimizador


Dispositivo utilizado: cpu


In [6]:
#aremos un bucle para las epocas
for epoch in range(num_epochs): #el numero de epocas
    for batch_idx, (data, targets) in enumerate(train_loader):#los datos de entrenamiento
        # Mover datos a CUDA si es posible
        data = data.to(device=device)#se mueven los datos de lote
        targets = targets.to(device=device) #s mueven las etiquetas del lote

        # Reorganizar los datos al formato correcto
        data = data.reshape(data.shape[0], -1)

        # Propagación hacia adelante
        scores = model(data)# paso datos para obtener mis predicciones
        loss = criterion(scores, targets)#calculo la perdida de las predicciones

        # Retropropagación
        optimizer.zero_grad() #se reinician las gradients para el optimizador
        loss.backward() #se calcula otra ves las gradientes por retropopagaion

        # Paso de descenso de gradiente o adam
        optimizer.step()

    # Imprimir la pérdida después de cada época
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/10], Loss: 0.0066
Epoch [2/10], Loss: 0.0020
Epoch [3/10], Loss: 0.0009
Epoch [4/10], Loss: 0.0006
Epoch [5/10], Loss: 0.0005
Epoch [6/10], Loss: 0.0003
Epoch [7/10], Loss: 0.0002
Epoch [8/10], Loss: 0.0001
Epoch [9/10], Loss: 0.0002
Epoch [10/10], Loss: 0.0003


In [7]:
def check_accuracy(loader, model): #toma los valores del modelo y los datos de x
    correct = 0  #variable de correctos
    total = 0   #variable del total
    model.eval()  #desactiva el comportamiento de capaz
    predictions = [] #variable de prediicones

    with torch.no_grad():
        for inputs, labels in loader: #se itera sobre el lote de datos
            inputs, labels = inputs.to(device), labels.to(device)# se al gpu

            # Forward pass
            outputs = model(inputs) #se obtiene las predicciones del modelo de entrada
            predicted = torch.round(outputs)  # Redondear las predicciones a 0 o 1

            # Contar predicciones correctas
            correct += (predicted == labels).sum().item()

            # Almacenar predicciones para análisis posterior si es necesario
            predictions.append(predicted.cpu().numpy())

            total += labels.size(0)

    model.train()#se vuelve a colocar el modelo de entramiento por si se entrena otra ves
    accuracy = correct / total
    return accuracy, predictions #precicion calculada

# Calcular la precisión en el conjunto de entrenamiento y prueba
train_accuracy, train_predictions = check_accuracy(train_loader, model)
test_accuracy, test_predictions = check_accuracy(test_loader, model)

print(f"precision en conjunto de entrenamiento: {train_accuracy*100:.2f}%")
print(f"precision en el conjunto de pruebat: {test_accuracy*100:.2f}%")


precision en conjunto de entrenamiento: 99.97%
precision en el conjunto de pruebat: 99.95%


EXTRAS PARA ALGUNOS CALCULOS POR E_JEMLO PERDIDA de cada epoca  y total ceros y unos despues de epocas

In [8]:
# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Mover datos a CUDA si es posible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # Reorganizar los datos al formato correcto
        data = data.reshape(data.shape[0], -1)

        # Propagación hacia adelante
        scores = model(data)
        predictions = torch.round(scores)  # Redondear las predicciones a 0 o 1

        # Imprimir las predicciones para cada lote
        print("Predictions:", predictions)

        loss = criterion(scores, targets)

        # Retropropagación
        optimizer.zero_grad()
        loss.backward()

        # Paso de descenso de gradiente o adam
        optimizer.step()

    # Imprimir la pérdida después de cada época
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Se han truncado las últimas 5000 líneas del flujo de salida.
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.]], grad_fn=<RoundBackward0>)
Predictions: tensor([[0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
   

In [9]:
# Train Network
for epoch in range(num_epochs):
    # Inicializar contadores para las clases 0 y 1
    total_zeros = 0
    total_ones = 0

    for batch_idx, (data, targets) in enumerate(train_loader):
        # Mover datos a CUDA si es posible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # Reorganizar los datos al formato correcto
        data = data.reshape(data.shape[0], -1)

        # Propagación hacia adelante
        scores = model(data)
        predictions = torch.round(scores)  # Redondear las predicciones a 0 o 1

        # Contar el número de predicciones de cada clase
        total_zeros += (predictions == 0).sum().item()
        total_ones += (predictions == 1).sum().item()

        loss = criterion(scores, targets)

        # Retropropagación
        optimizer.zero_grad()
        loss.backward()

        # Paso de descenso de gradiente o adam
        optimizer.step()

    # Imprimir el total de predicciones de cada clase después de cada época
    print(f'Epoch [{epoch+1}/{num_epochs}], Total zeros: {total_zeros}, Total ones: {total_ones}, Loss: {loss.item():.4f}')


Epoch [1/10], Total zeros: 23507, Total ones: 23369, Loss: 0.0001
Epoch [2/10], Total zeros: 23504, Total ones: 23372, Loss: 0.0001
Epoch [3/10], Total zeros: 23505, Total ones: 23371, Loss: 0.0000
Epoch [4/10], Total zeros: 23505, Total ones: 23371, Loss: 0.0000
Epoch [5/10], Total zeros: 23507, Total ones: 23369, Loss: 0.0000
Epoch [6/10], Total zeros: 23505, Total ones: 23371, Loss: 0.0000
Epoch [7/10], Total zeros: 23507, Total ones: 23369, Loss: 0.0000
Epoch [8/10], Total zeros: 23506, Total ones: 23370, Loss: 0.0000
Epoch [9/10], Total zeros: 23506, Total ones: 23370, Loss: 0.0000
Epoch [10/10], Total zeros: 23508, Total ones: 23368, Loss: 0.0002
